In [1]:
from multiprocessing import Pool
import pandas as pd
import numpy as np
import allel
import pysam
import json

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
refgene = pd.read_csv('data/refseq-20200109.txt', sep='\t')
refgene['length'] = refgene['cdsEnd'] - refgene['cdsStart']
refgene = refgene.sort_values('length', ascending=False)
refgene = refgene.drop_duplicates('name2', keep='first')

In [4]:
refgene.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames,length
21052,26,NM_014141.5,chr7,+,145813452,148118090,145813968,148112708,24,"145813452,146471362,146536802,146740998,146805...","145814065,146471473,146536996,146741146,146805...",0,CNTNAP2,cmpl,cmpl,"0,1,1,0,1,1,0,0,1,1,2,1,1,1,2,1,1,1,1,1,0,1,1,1,",2298740
27890,12,NM_000109.3,chrX,-,31137344,33357726,31140035,33357382,79,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",0,DMD,cmpl,cmpl,"0,1,1,0,2,2,2,2,2,0,2,0,1,2,1,1,2,1,0,0,1,0,2,...",2217347
33217,2,NM_001142699.1,chr11,-,83166054,85338314,83170860,85309740,28,"83166054,83173044,83177750,83180243,83182668,8...","83170967,83173136,83177860,83180416,83182770,8...",0,DLG2,cmpl,cmpl,"1,2,0,1,1,1,0,2,2,1,0,2,2,2,1,0,1,2,0,0,0,0,0,...",2138880
23572,9,NM_033225.5,chr8,-,2792874,4852328,2796106,4851938,70,"2792874,2799993,2806820,2807752,2808635,281174...","2796266,2800126,2806908,2807865,2808797,281179...",0,CSMD1,cmpl,cmpl,"2,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",2055832
54718,10,NM_080676.5,chr20,+,13976145,16033842,13976409,16030521,17,"13976145,13982933,14066266,14474124,14665488,1...","13976455,13983050,14066374,14474154,14665605,1...",0,MACROD2,cmpl,cmpl,"0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,",2054112


In [5]:
gene_info = pd.read_csv('data/Homo_sapiens.gene_info.gz', sep='\t')

In [6]:
gene_info.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20191220,-
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20200113,-
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000256069,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20191221,-
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171428,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20200209,-
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156006,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20200121,-


In [7]:
refgene_info = refgene.merge(gene_info, left_on='name2', right_on='Symbol')

In [8]:
refgene_info.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames,length,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,26,NM_014141.5,chr7,+,145813452,148118090,145813968,148112708,24,"145813452,146471362,146536802,146740998,146805...","145814065,146471473,146536996,146741146,146805...",0,CNTNAP2,cmpl,cmpl,"0,1,1,0,1,1,0,0,1,1,2,1,1,1,2,1,1,1,1,1,0,1,1,1,",2298740,9606,26047,CNTNAP2,-,AUTS15|CASPR2|CDFE|NRXN4|PTHSL1,MIM:604569|HGNC:HGNC:13830|Ensembl:ENSG0000017...,7,7q35-q36.1,contactin associated protein 2,protein-coding,CNTNAP2,contactin associated protein 2,O,contactin-associated protein-like 2|cell recog...,20200105,-
1,12,NM_000109.3,chrX,-,31137344,33357726,31140035,33357382,79,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",0,DMD,cmpl,cmpl,"0,1,1,0,2,2,2,2,2,0,2,0,1,2,1,1,2,1,0,0,1,0,2,...",2217347,9606,1756,DMD,-,BMD|CMD3B|DXS142|DXS164|DXS206|DXS230|DXS239|D...,MIM:300377|HGNC:HGNC:2928|Ensembl:ENSG00000198947,X,Xp21.2-p21.1,dystrophin,protein-coding,DMD,dystrophin,O,dystrophin,20200210,-
2,2,NM_001142699.1,chr11,-,83166054,85338314,83170860,85309740,28,"83166054,83173044,83177750,83180243,83182668,8...","83170967,83173136,83177860,83180416,83182770,8...",0,DLG2,cmpl,cmpl,"1,2,0,1,1,1,0,2,2,1,0,2,2,2,1,0,1,2,0,0,0,0,0,...",2138880,9606,1740,DLG2,-,PPP1R58|PSD-93|PSD93|chapsyn-110,MIM:603583|HGNC:HGNC:2901|Ensembl:ENSG00000150672,11,11q14.1,discs large MAGUK scaffold protein 2,protein-coding,DLG2,discs large MAGUK scaffold protein 2,O,disks large homolog 2|DLG2 protein isoform|cha...,20200209,-
3,9,NM_033225.5,chr8,-,2792874,4852328,2796106,4851938,70,"2792874,2799993,2806820,2807752,2808635,281174...","2796266,2800126,2806908,2807865,2808797,281179...",0,CSMD1,cmpl,cmpl,"2,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",2055832,9606,64478,CSMD1,-,PPP1R24,MIM:608397|HGNC:HGNC:14026|Ensembl:ENSG0000018...,8,8p23.2,CUB and Sushi multiple domains 1,protein-coding,CSMD1,CUB and Sushi multiple domains 1,O,CUB and sushi domain-containing protein 1|prot...,20200209,-
4,10,NM_080676.5,chr20,+,13976145,16033842,13976409,16030521,17,"13976145,13982933,14066266,14474124,14665488,1...","13976455,13983050,14066374,14474154,14665605,1...",0,MACROD2,cmpl,cmpl,"0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,",2054112,9606,140733,MACROD2,-,C20orf133|C2orf133,MIM:611567|HGNC:HGNC:16126|Ensembl:ENSG0000017...,20,20p12.1,mono-ADP ribosylhydrolase 2,protein-coding,MACROD2,mono-ADP ribosylhydrolase 2,O,ADP-ribose glycohydrolase MACROD2|MACRO domain...,20191221,-


In [9]:
gene = refgene_info.loc[
    refgene_info['type_of_gene'] == 'protein-coding',
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [10]:
gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
17803,chr1,69090,70008,79501,OR4F5,NM_001005484.1,+
17672,chr1,367658,368597,729759,OR4F29,NM_001005221.2,+
17664,chr1,621095,622034,81399,OR4F16,NM_001005277.1,-
9248,chr1,861120,879961,148398,SAMD11,NM_152486.2,+
10208,chr1,879582,894679,26155,NOC2L,NM_015658.3,-


In [11]:
gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/gene-20200109.sorted.bed', index=False, sep='\t')

In [12]:
!bgzip -cf data/gene-20200109.sorted.bed > ../data/gene-20200109.sorted.bed.gz

In [13]:
!tabix -fp bed ../data/gene-20200109.sorted.bed.gz

In [14]:
with open('data/omim-gene.list') as f:
    omim_gene = set(f.read().split('\n'))

In [15]:
omim_gene = refgene_info.loc[
    refgene_info['name2'].isin(omim_gene),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [16]:
omim_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
17825,chr1,948846,949920,9636,ISG15,NM_005101.3,+
6298,chr1,955502,991499,375790,AGRN,NM_001305275.1,+
15599,chr1,1146705,1149548,7293,TNFRSF4,NM_003327.3,-
17342,chr1,1167628,1170421,126792,B3GALT6,NM_080605.3,+
10735,chr1,1270657,1284492,1855,DVL1,NM_001330311.1,-


In [17]:
omim_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/omim-gene-20200109.sorted.bed', index=False, sep='\t')

In [18]:
!bgzip -cf data/omim-gene-20200109.sorted.bed > ../data/omim-gene-20200109.sorted.bed.gz

In [19]:
!tabix -fp bed ../data/omim-gene-20200109.sorted.bed.gz

In [20]:
curation_gene = pd.read_csv('data/ClinGen_gene_curation_list_GRCh37.tsv', sep='\t', dtype=str, skiprows=5)

In [21]:
curation_gene.head()

,#Gene Symbol,Gene ID,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID
0,A4GALT,53947,22q13.2,chr22:43088121-43117307,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,2014-12-11,111400,NaN
1,AAGAB,79719,15q23,chr15:67493013-67547536,3,Sufficient evidence for dosage pathogenicity,23064416,23000146,NaN,0,No evidence available,NaN,NaN,NaN,2013-02-28,148600,NaN
2,AARS,16,16q22.1,chr16:70286297-70323412,0,No evidence available,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,2018-01-11,NaN,NaN
3,AARS2,57505,6p21.1,chr6:44266463-44281063,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,2016-08-22,NaN,NaN
4,AASS,10157,7q31.32,chr7:121713598-121784344,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,2016-08-22,NaN,NaN


In [22]:
hi_genes = set(
    curation_gene.loc[
        curation_gene['Haploinsufficiency Score'] == '3', '#Gene Symbol'
    ]
)

In [23]:
hi_gene = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [24]:
hi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
47,chr1,6845383,7829766,23261,CAMTA1,NM_015215.3,+
6265,chr1,17345216,17380665,6390,SDHB,NM_003000.2,-
2946,chr1,27022521,27108601,8289,ARID1A,NM_006015.4,+
14219,chr1,27860755,27930005,27245,AHDC1,NM_001029882.3,-
6768,chr1,43391045,43424847,6513,SLC2A1,NM_006516.2,-


In [25]:
hi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-gene-20200109.sorted.bed', index=False, sep='\t')

In [26]:
!bgzip -cf data/hi-gene-20200109.sorted.bed > ../data/hi-gene-20200109.sorted.bed.gz

In [27]:
!tabix -fp bed ../data/hi-gene-20200109.sorted.bed.gz

In [28]:
hi_cds = refgene_info.loc[
    (refgene_info['name2'].isin(hi_genes)) & (refgene_info['length'] != 0),
    ['chrom', 'cdsStart', 'cdsEnd', 'GeneID', 'name2', 'name']
].sort_values(['chrom', 'cdsStart', 'cdsEnd'])

In [29]:
hi_cds.head()

,chrom,cdsStart,cdsEnd,GeneID,name2,name
47,chr1,6845590,7826551,23261,CAMTA1,NM_015215.3
6265,chr1,17345375,17380514,6390,SDHB,NM_003000.2
2946,chr1,27022894,27107247,8289,ARID1A,NM_006015.4
14219,chr1,27873814,27878626,27245,AHDC1,NM_001029882.3
6768,chr1,43392711,43424322,6513,SLC2A1,NM_006516.2


In [30]:
hi_cds.rename(columns={
    'chrom': '#chrom', 'cdsStart': 'start', 'cdsEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-cds-20200109.sorted.bed', index=False, sep='\t')

In [31]:
!bgzip -cf data/hi-cds-20200109.sorted.bed > ../data/hi-cds-20200109.sorted.bed.gz

In [32]:
!tabix -fp bed ../data/hi-cds-20200109.sorted.bed.gz

In [33]:
hi_exons = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes), ['chrom', 'exonStarts', 'exonEnds', 'GeneID', 'name2', 'name', 'strand']
].copy()

In [34]:
hi_exons.head()

,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
1,chrX,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",1756,DMD,NM_000109.3,-
23,chr7,"69063904,69364271,69583117,69599521,69900737,7...","69064948,69364484,69583219,69599557,69900767,7...",26053,AUTS2,NM_015570.3,+
27,chrX,"28605680,28807436,29301054,29414374,29417271,2...","28606164,28807542,29301334,29414561,29417425,2...",11141,IL1RAPL1,NM_014271.3,+
40,chr2,"50145642,50170850,50280408,50318460,50463926,5...","50149389,50170929,50280728,50318632,50464108,5...",9378,NRXN1,NM_001330096.1,-
47,chr1,"6845383,6880240,6885151,7151363,7309550,752788...","6845635,6880310,6885270,7151431,7309686,752796...",23261,CAMTA1,NM_015215.3,+


In [35]:
hi_exons['exonStarts'] = hi_exons['exonStarts'].str.replace(r',$', '')
hi_exons['exonEnds'] = hi_exons['exonEnds'].str.replace(r',$', '')

In [36]:
hi_exons.head()

,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
1,chrX,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",1756,DMD,NM_000109.3,-
23,chr7,"69063904,69364271,69583117,69599521,69900737,7...","69064948,69364484,69583219,69599557,69900767,7...",26053,AUTS2,NM_015570.3,+
27,chrX,"28605680,28807436,29301054,29414374,29417271,2...","28606164,28807542,29301334,29414561,29417425,2...",11141,IL1RAPL1,NM_014271.3,+
40,chr2,"50145642,50170850,50280408,50318460,50463926,5...","50149389,50170929,50280728,50318632,50464108,5...",9378,NRXN1,NM_001330096.1,-
47,chr1,"6845383,6880240,6885151,7151363,7309550,752788...","6845635,6880310,6885270,7151431,7309686,752796...",23261,CAMTA1,NM_015215.3,+


In [37]:
start = hi_exons['exonStarts'].str.split(',').apply(pd.Series).stack().reset_index()
start = start.rename(columns={'level_0': 'row', 0: 'start'})[['row', 'start']]
start['start'] = start['start'].astype(int)
end = hi_exons['exonEnds'].str.split(',').apply(pd.Series).stack().reset_index()
end = end.rename(columns={0: 'end'})['end'].astype(int)
position = start.join(end)

In [38]:
position.head()

,row,start,end
0,1,31137344,31140047
1,1,31144758,31144790
2,1,31152218,31152311
3,1,31164407,31164531
4,1,31165391,31165635


In [39]:
exon = position.merge(
    hi_exons[['chrom', 'GeneID', 'name2', 'name', 'strand']], how='left', left_on='row', right_index=True
)
exon = exon.sort_values(['chrom', 'start', 'end'])

In [40]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand
126,47,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+
127,47,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+
128,47,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+
129,47,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+
130,47,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+


In [41]:
exon['+'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=True).astype(int)
exon['-'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=False).astype(int)

In [42]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-
126,47,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23
127,47,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22
128,47,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21
129,47,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20
130,47,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19


In [43]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-
3759,7339,545352,545561,chrY,6473,SHOX,NM_006883.2,+,3,4
3760,7339,551555,551613,chrY,6473,SHOX,NM_006883.2,+,4,3
3761,7339,551733,551822,chrY,6473,SHOX,NM_006883.2,+,5,2
3762,7339,569519,570146,chrY,6473,SHOX,NM_006883.2,+,6,1
4615,18161,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1


In [44]:
exon['exon'] = pd.concat([exon.loc[exon['strand'] == '+', '+'], exon.loc[exon['strand'] == '-', '-']])

In [45]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon
126,47,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23,1
127,47,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22,2
128,47,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21,3
129,47,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20,4
130,47,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19,5


In [46]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon
3759,7339,545352,545561,chrY,6473,SHOX,NM_006883.2,+,3,4,3
3760,7339,551555,551613,chrY,6473,SHOX,NM_006883.2,+,4,3,4
3761,7339,551733,551822,chrY,6473,SHOX,NM_006883.2,+,5,2,5
3762,7339,569519,570146,chrY,6473,SHOX,NM_006883.2,+,6,1,6
4615,18161,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1,1


In [47]:
exon['last_exon'] = exon.groupby(['name2', 'name'])['exon'].transform('max') == exon['exon']

In [48]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon,last_exon
126,47,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23,1,False
127,47,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22,2,False
128,47,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21,3,False
129,47,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20,4,False
130,47,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19,5,False


In [49]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon,last_exon
3759,7339,545352,545561,chrY,6473,SHOX,NM_006883.2,+,3,4,3,False
3760,7339,551555,551613,chrY,6473,SHOX,NM_006883.2,+,4,3,4,False
3761,7339,551733,551822,chrY,6473,SHOX,NM_006883.2,+,5,2,5,False
3762,7339,569519,570146,chrY,6473,SHOX,NM_006883.2,+,6,1,6,True
4615,18161,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1,1,True


In [50]:
exon = exon[
    ['chrom', 'start', 'end', 'GeneID', 'name2', 'name', 'exon', 'last_exon']
].sort_values(['chrom', 'start', 'end'])

In [51]:
exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
126,chr1,6845383,6845635,23261,CAMTA1,NM_015215.3,1,False
127,chr1,6880240,6880310,23261,CAMTA1,NM_015215.3,2,False
128,chr1,6885151,6885270,23261,CAMTA1,NM_015215.3,3,False
129,chr1,7151363,7151431,23261,CAMTA1,NM_015215.3,4,False
130,chr1,7309550,7309686,23261,CAMTA1,NM_015215.3,5,False


In [52]:
exon.rename(columns={
    'chrom': '#chrom',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-exon-20200109.sorted.bed', index=False, sep='\t')

In [53]:
!bgzip -cf data/hi-exon-20200109.sorted.bed > ../data/hi-exon-20200109.sorted.bed.gz

In [54]:
!tabix -fp bed ../data/hi-exon-20200109.sorted.bed.gz

In [55]:
last_exon = exon[exon['last_exon']]

In [56]:
last_exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
148,chr1,7826518,7829766,23261,CAMTA1,NM_015215.3,23,True
3443,chr1,17345216,17345453,6390,SDHB,NM_003000.2,8,True
2226,chr1,27105513,27108601,8289,ARID1A,NM_006015.4,20,True
4530,chr1,27860755,27861427,27245,AHDC1,NM_001029882.3,7,True
3646,chr1,43391045,43392912,6513,SLC2A1,NM_006516.2,10,True


In [57]:
last_exon_region = last_exon['chrom'] + ':' + last_exon['start'].astype(str) + '-' + last_exon['end'].astype(str)
last_exon_region = last_exon_region.str.replace('chr', '')

In [58]:
last_exon_region.head()

148       1:7826518-7829766
3443    1:17345216-17345453
2226    1:27105513-27108601
4530    1:27860755-27861427
3646    1:43391045-43392912
dtype: object

In [59]:
need_fields = [
    'variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT',
    'variants/AF_ESP', 'variants/AF_EXAC', 'variants/AF_TGP', 'variants/CLNSIG'
]

In [60]:
with open('data/clinvar_pathogenic_20191125.sorted.vcf', 'w') as f:
    headers = allel.read_vcf_headers('data/clinvar_20191125.vcf.gz')
    f.write(''.join(headers.headers))
    
    def fetch_variants(region):
        fields, samples, headers, it = allel.iter_vcf_chunks(
            'data/clinvar_20191125.vcf.gz', fields=need_fields, alt_number=1, region=region
        )
        for variants, *_ in it:
            esp_filter = np.isnan(variants['variants/AF_ESP'])
            esp_filter[~esp_filter] |= variants['variants/AF_ESP'][~esp_filter] < 0.01

            exac_filter = np.isnan(variants['variants/AF_EXAC'])
            exac_filter[~exac_filter] |= variants['variants/AF_EXAC'][~exac_filter] < 0.01

            tgp_filter = np.isnan(variants['variants/AF_TGP'])
            tgp_filter[~tgp_filter] |= variants['variants/AF_TGP'][~tgp_filter] < 0.01

            pathogenic_filter = np.isin(
                variants['variants/CLNSIG'], ['Likely_pathogenic', 'Pathogenic', 'Pathogenic/Likely_pathogenic']
            )

            af_filter = esp_filter & exac_filter & tgp_filter & pathogenic_filter

            filtered_variants = {k: v[af_filter] for k, v in variants.items()}

            filtered_variants['variants/CHROM'] = 'chr' + filtered_variants['variants/CHROM']

            return allel.normalize_callset(filtered_variants)
    
    with Pool(processes=7) as pool:
        variants = pool.map(fetch_variants, last_exon_region)
    
    for names, callset in filter(lambda x: x is not None, variants):
        allel.write_vcf_data(f, names, callset, None, {field: np.nan for field in need_fields})

In [61]:
!sed -i 's/AF_.\+=nan;//' data/clinvar_pathogenic_20191125.sorted.vcf

In [62]:
!bgzip -cf data/clinvar_pathogenic_20191125.sorted.vcf > ../data/clinvar-pathogenic-20191125.sorted.vcf.gz

In [63]:
!tabix -fp vcf ../data/clinvar-pathogenic-20191125.sorted.vcf.gz

In [64]:
uhi_genes = set(
    curation_gene.loc[curation_gene['Haploinsufficiency Score'] == '40', '#Gene Symbol']
)

In [65]:
uhi_gene = refgene_info.loc[
    refgene_info['name2'].isin(uhi_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [66]:
uhi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
8064,chr1,55505148,55530526,255738,PCSK9,NM_174936.3,+
18085,chr13,20761601,20767114,2706,GJB2,NM_004004.5,-


In [67]:
uhi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/uhi-gene-20200109.sorted.bed', index=False, sep='\t')

In [68]:
!bgzip -cf data/uhi-gene-20200109.sorted.bed > ../data/uhi-gene-20200109.sorted.bed.gz

In [69]:
!tabix -fp bed ../data/uhi-gene-20200109.sorted.bed.gz

In [70]:
region = pd.read_csv('data/ClinGen_region_curation_list_GRCh37.tsv', sep='\t', skiprows=5, dtype=str)

In [71]:
region.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID
0,ISCA-46300,15q24 recurrent region (C-D) (includes SIN3A),NaN,chr15: 75631787-75972909,3,Sufficient evidence for dosage pathogenicity,27399968,22180641,NaN,0,No evidence available,NaN,NaN,NaN,2020-01-13,613406,NaN
1,ISCA-46299,Xp11.22 region (includes HUWE1),Xp11.22,chrX:53363456-53793054,0,No evidence available,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,22840365,20655035,26692240,2018-11-19,NaN,NaN
2,ISCA-46298,13q12.12 recurrent region (includes SACS and S...,13q12.12,chr13:23544582-24881361,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,23239789,NaN,NaN,2019-11-03,"270550,253700",NaN
3,ISCA-46297,16p12.2 recurrent region (includes OTOA) (dist...,16p12.2,chr16:21570113-21740423,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,40,Dosage sensitivity unlikely,NaN,NaN,NaN,2019-11-03,607039,NaN
4,ISCA-46296,15q24 recurrent region (A-C),15q24.1-q24.2,chr15:72963715-75508312,3,Sufficient evidence for dosage pathogenicity,22180641,19921647,NaN,1,Little evidence for dosage pathogenicity,19557438,20860070,NaN,2020-01-03,NaN,NaN


In [72]:
position = region['Genomic Location'].str.extract(r'(?P<chrom>chr\w+)\s*:\s*(?P<start>\d+)\s*-\s*(?P<end>\d+)')
position['start'] = position['start'].astype(int)
position['end'] = position['end'].astype(int)

In [73]:
position.head()

,chrom,start,end
0,chr15,75631787,75972909
1,chrX,53363456,53793054
2,chr13,23544582,24881361
3,chr16,21570113,21740423
4,chr15,72963715,75508312


In [74]:
region_pos = region.merge(position, how='left', left_index=True, right_index=True)

In [75]:
region_pos.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID,chrom,start,end
0,ISCA-46300,15q24 recurrent region (C-D) (includes SIN3A),NaN,chr15: 75631787-75972909,3,Sufficient evidence for dosage pathogenicity,27399968,22180641,NaN,0,No evidence available,NaN,NaN,NaN,2020-01-13,613406,NaN,chr15,75631787,75972909
1,ISCA-46299,Xp11.22 region (includes HUWE1),Xp11.22,chrX:53363456-53793054,0,No evidence available,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,22840365,20655035,26692240,2018-11-19,NaN,NaN,chrX,53363456,53793054
2,ISCA-46298,13q12.12 recurrent region (includes SACS and S...,13q12.12,chr13:23544582-24881361,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,23239789,NaN,NaN,2019-11-03,"270550,253700",NaN,chr13,23544582,24881361
3,ISCA-46297,16p12.2 recurrent region (includes OTOA) (dist...,16p12.2,chr16:21570113-21740423,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,40,Dosage sensitivity unlikely,NaN,NaN,NaN,2019-11-03,607039,NaN,chr16,21570113,21740423
4,ISCA-46296,15q24 recurrent region (A-C),15q24.1-q24.2,chr15:72963715-75508312,3,Sufficient evidence for dosage pathogenicity,22180641,19921647,NaN,1,Little evidence for dosage pathogenicity,19557438,20860070,NaN,2020-01-03,NaN,NaN,chr15,72963715,75508312


In [76]:
hi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [77]:
hi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
43,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD)
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
17,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3)
51,chr10,81682843,88739388,ISCA-37424,10q22.3q23.2 recurrent region (LCR-3/4-flanked...
63,chr11,31803509,32510988,ISCA-37401,11p13 (WAGR syndrome) region


In [78]:
hi_region['omim_genes'] = hi_region.apply(
    lambda row: ','.join(omim_gene.loc[
        (omim_gene['chrom'] == row['chrom']) & (omim_gene['txStart'] >= row['start']) & (omim_gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [79]:
hi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,omim_genes
43,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD),"ISG15,AGRN,TNFRSF4,B3GALT6,DVL1,ATAD3A,TMEM240..."
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc...","GJA5,GJA8"
17,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3),"SDCCAG8,AKT3,ZBTB18,COX20,HNRNPU"
51,chr10,81682843,88739388,ISCA-37424,10q22.3q23.2 recurrent region (LCR-3/4-flanked...,"ANXA11,MAT1A,CDHR1,RGR,LDB3,BMPR1A"
63,chr11,31803509,32510988,ISCA-37401,11p13 (WAGR syndrome) region,"PAX6,WT1"


In [80]:
hi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/hi-region-20200109.sorted.bed', sep='\t', index=False)

In [81]:
!bgzip -cf data/hi-region-20200109.sorted.bed > ../data/hi-region-20200109.sorted.bed.gz

In [82]:
!tabix -fp bed ../data/hi-region-20200109.sorted.bed.gz

In [83]:
uhi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [84]:
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region)
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region)
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere
21,chr16,34202088,35147508,ISCA-37481,3 copies: 16p centromere
20,chr19,43242796,43741310,ISCA-37483,19q13.3 region (PSG gene cluster)


In [85]:
uhi_region['genes'] = uhi_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom']) & (gene['txStart'] >= row['start']) & (gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [86]:
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region),"FAM72C,PPIAL4C,FCGR1A"
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region),"OR4E2,OR4E1"
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere,
21,chr16,34202088,35147508,ISCA-37481,3 copies: 16p centromere,
20,chr19,43242796,43741310,ISCA-37483,19q13.3 region (PSG gene cluster),"PSG8,PSG1,PSG6,PSG7,PSG11,PSG2,PSG5,PSG4"


In [87]:
uhi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/uhi-region-20200109.sorted.bed', sep='\t', index=False)

In [88]:
!bgzip -cf data/uhi-region-20200109.sorted.bed > ../data/uhi-region-20200109.sorted.bed.gz

In [89]:
!tabix -fp bed ../data/uhi-region-20200109.sorted.bed.gz

In [90]:
with open('data/gene-table.json') as f:
    data = json.load(f)
    decipher = pd.DataFrame.from_records(data, columns=['name', 'hi_score', 'p_li']).rename(columns={'name': 'symbol'})
    decipher = decipher.merge(gene, left_on='symbol', right_on='name2')

In [91]:
decipher.head()

,symbol,hi_score,p_li,chrom,txStart,txEnd,GeneID,name2,name,strand
0,A2ML1,0.011664,1.329022e-22,chr12,8975067,9029379,144568,A2ML1,NM_144670.5,+
1,AARS2,0.089486,1.648580e-07,chr6,44266462,44281063,57505,AARS2,NM_020745.3,-
2,AASS,0.132071,1.355315e-06,chr7,121713597,121784344,10157,AASS,NM_005763.3,-
3,ABCA10,0.002180,1.206886e-27,chr17,67144147,67240956,10349,ABCA10,NM_080282.3,-
4,ABCA13,0.047218,3.307328e-78,chr7,48211056,48687091,154664,ABCA13,NM_152701.4,+


In [92]:
gnomad = pd.read_csv('data/gnomad.v2.1.1.lof_metrics.by_gene.txt', sep='\t', index_col=0)

In [93]:
gnomad.head()

,transcript,obs_mis,exp_mis,oe_mis,mu_mis,possible_mis,obs_mis_pphen,exp_mis_pphen,oe_mis_pphen,possible_mis_pphen,obs_syn,exp_syn,oe_syn,mu_syn,possible_syn,obs_lof,mu_lof,possible_lof,exp_lof,pLI,pNull,pRec,oe_lof,oe_syn_lower,oe_syn_upper,oe_mis_lower,oe_mis_upper,oe_lof_lower,oe_lof_upper,constraint_flag,syn_z,mis_z,lof_z,oe_lof_upper_rank,oe_lof_upper_bin,oe_lof_upper_bin_6,n_sites,classic_caf,max_af,no_lofs,obs_het_lof,obs_hom_lof,defined,p,exp_hom_lof,classic_caf_afr,classic_caf_amr,classic_caf_asj,classic_caf_eas,classic_caf_fin,classic_caf_nfe,classic_caf_oth,classic_caf_sas,p_afr,p_amr,p_asj,p_eas,p_fin,p_nfe,p_oth,p_sas,transcript_type,gene_id,transcript_level,cds_length,num_coding_exons,gene_type,gene_length,exac_pLI,exac_obs_lof,exac_exp_lof,exac_oe_lof,brain_expression,chromosome,start_position,end_position
gene,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MED13,ENST00000397786,871,1117.80,0.77921,0.000056,14195,314.0,529.75,0.59273,6708.0,422,387.53,1.0890,0.000019,4248,0.0,0.000005,1257.0,98.429,1.0,8.943600e-40,1.838300e-16,0.000000,1.005,1.180,0.736,0.824,0.000,0.030,NaN,-1.376500,2.6232,9.1935,0.0,0.0,0.0,2.0,0.000012,0.000008,124782.0,3.0,0.0,124785.0,0.000012,0.000018,0.000000,0.000000,0.000000,0.000000,0.000093,0.000009,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000093,0.000009,0.0,0.000000,protein_coding,ENSG00000108510,2,6522,30,protein_coding,122678,1.0,0.0,64.393,0.000000,NaN,17,60019966,60142643
NIPBL,ENST00000282516,846,1441.50,0.58688,0.000074,18540,158.0,543.10,0.29092,7135.0,496,495.01,1.0020,0.000025,5211,1.0,0.000009,1781.0,150.320,1.0,2.977300e-59,3.572400e-24,0.006653,0.930,1.079,0.554,0.621,0.001,0.032,NaN,-0.035119,5.5737,11.2860,1.0,0.0,0.0,2.0,0.000012,0.000008,125693.0,3.0,0.0,125696.0,0.000012,0.000018,0.000000,0.000000,0.000099,0.000000,0.000000,0.000000,0.0,0.000065,0.000000,0.000000,0.000099,0.000000,0.000000,0.000000,0.0,0.000065,protein_coding,ENSG00000164190,2,8412,46,protein_coding,189655,1.0,1.0,110.570,0.009044,NaN,5,36876861,37066515
SMC3,ENST00000361804,178,630.07,0.28251,0.000032,8109,21.0,182.52,0.11506,2197.0,215,203.25,1.0578,0.000010,2091,0.0,0.000005,937.0,79.490,1.0,2.785300e-32,2.191400e-13,0.000000,0.946,1.184,0.249,0.320,0.000,0.037,NaN,-0.647760,6.3999,8.2618,2.0,0.0,0.0,8.0,0.000032,0.000004,125731.0,8.0,0.0,125739.0,0.000032,0.000127,0.000000,0.000000,0.000099,0.000054,0.000000,0.000044,0.0,0.000033,0.000000,0.000000,0.000099,0.000054,0.000000,0.000044,0.0,0.000033,protein_coding,ENSG00000108055,2,3651,29,protein_coding,36946,1.0,0.0,58.523,0.000000,NaN,10,112327449,112364394
CNOT1,ENST00000317147,561,1295.90,0.43290,0.000069,15670,51.0,290.68,0.17545,3560.0,470,456.03,1.0306,0.000024,4564,1.0,0.000007,1440.0,125.030,1.0,2.992400e-49,4.562900e-20,0.007998,0.955,1.112,0.403,0.464,0.002,0.038,NaN,-0.514100,7.2546,10.2790,3.0,0.0,0.0,5.0,0.000020,0.000004,125740.0,4.0,0.0,125744.0,0.000016,0.000032,0.000000,0.000029,0.000000,0.000055,0.000000,0.000026,0.0,0.000000,0.000000,0.000029,0.000000,0.000054,0.000000,0.000018,0.0,0.000000,protein_coding,ENSG00000125107,2,7128,48,protein_coding,109936,1.0,3.0,90.130,0.033285,NaN,16,58553855,58663790
RLF,ENST00000372771,669,972.87,0.68766,0.000047,12682,107.0,321.14,0.33319,4151.0,358,352.62,1.0153,0.000017,3482,0.0,0.000004,1024.0,73.222,1.0,8.405500e-30,2.284200e-12,0.000000,0.930,1.108,0.645,0.733,0.000,0.040,NaN,-0.225180,3.4620,7.9294,4.0,0.0,0.0,1.0,0.000004,0.000004,125122.0,1.0,0.0,125123.0,0.000004,0.000002,0.000062,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000062,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,protein_coding,ENSG00000117000,2,5742,8,protein_coding,79549,1.0,0.0,43.607,0.000000,NaN,1,40627045,40706593


In [94]:
decipher = decipher.join(gnomad['pLI'], on='name2')

In [95]:
decipher.head()

,symbol,hi_score,p_li,chrom,txStart,txEnd,GeneID,name2,name,strand,pLI
0,A2ML1,0.011664,1.329022e-22,chr12,8975067,9029379,144568,A2ML1,NM_144670.5,+,1.610900e-34
1,AARS2,0.089486,1.648580e-07,chr6,44266462,44281063,57505,AARS2,NM_020745.3,-,4.977700e-17
2,AASS,0.132071,1.355315e-06,chr7,121713597,121784344,10157,AASS,NM_005763.3,-,2.310100e-12
3,ABCA10,0.002180,1.206886e-27,chr17,67144147,67240956,10349,ABCA10,NM_080282.3,-,7.659300e-34
4,ABCA13,0.047218,3.307328e-78,chr7,48211056,48687091,154664,ABCA13,NM_152701.4,+,8.775300e-114


In [96]:
decipher = decipher.loc[
    (decipher['pLI'] >= 0.9) & (decipher['hi_score'] < 0.1),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'pLI', 'hi_score']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [97]:
decipher.head()

,chrom,txStart,txEnd,GeneID,name2,name,pLI,hi_score
2112,chr1,2439969,2458067,55229,PANK4,NM_018216.2,0.98325,0.098966
2587,chr1,155290639,155300909,23623,RUSC1,NM_001105203.1,0.94130,0.035285
2705,chr1,232533710,232651354,57568,SIPA1L2,NM_020808.4,0.99215,0.079950
1608,chr10,852853,977645,23185,LARP4B,NM_015155.2,0.99998,0.071477
199,chr10,24872537,25012597,57584,ARHGAP21,NM_020824.3,1.00000,0.073197


In [98]:
decipher.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/decipher-gene-20200109.sorted.bed', sep='\t', index=False)

In [99]:
!bgzip -cf data/decipher-gene-20200109.sorted.bed > ../data/decipher-gene-20200109.sorted.bed.gz

In [100]:
!tabix -fp bed ../data/decipher-gene-20200109.sorted.bed.gz

In [101]:
ts_genes = set(
    curation_gene.loc[
        curation_gene['Triplosensitivity Score'] == '3', '#Gene Symbol'
    ]
)

In [102]:
ts_gene = refgene_info.loc[
    refgene_info['name2'].isin(ts_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [103]:
ts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
4186,chr5,126112314,126172712,4001,LMNB1,NM_005573.3,+
10499,chrX,103031433,103047548,5354,PLP1,NM_001128834.2,+
3662,chrX,153295684,153363188,4204,MECP2,NM_001110792.1,-


In [104]:
ts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/ts-gene-20200109.sorted.bed', index=False, sep='\t')

In [105]:
!bgzip -cf data/ts-gene-20200109.sorted.bed > ../data/ts-gene-20200109.sorted.bed.gz

In [106]:
!tabix -fp bed ../data/ts-gene-20200109.sorted.bed.gz

In [107]:
uts_genes = set(
    curation_gene.loc[curation_gene['Triplosensitivity Score'] == '40', '#Gene Symbol']
)

In [108]:
uts_gene = refgene_info.loc[
    refgene_info['name2'].isin(uts_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [109]:
uts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
17987,chrX,6451658,6453159,51481,VCX3A,NM_016379.3,-


In [110]:
uts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/uts-gene-20200109.sorted.bed', index=False, sep='\t')

In [111]:
!bgzip -cf data/uts-gene-20200109.sorted.bed > ../data/uts-gene-20200109.sorted.bed.gz

In [112]:
!tabix -fp bed ../data/uts-gene-20200109.sorted.bed.gz

In [113]:
ts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [114]:
ts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
62,chr15,22832519,28379874,ISCA-37404,"15q11q13 recurrent (PWS/AS) region (BP1-BP3, C..."
23,chr15,23747996,28379874,ISCA-37478,"15q11q13 recurrent (PWS/AS) region (BP2-BP3, C..."
64,chr16,29649997,30199852,ISCA-37400,"16p11.2 recurrent region (proximal, BP4-BP5) (..."
47,chr17,1247833,2588909,ISCA-37430,17p13.3 (Miller-Dieker syndrome) region (inclu...


In [115]:
ts_region['omim_genes'] = ts_region.apply(
    lambda row: ','.join(omim_gene.loc[
        (omim_gene['chrom'] == row['chrom']) & (omim_gene['txStart'] >= row['start']) & (omim_gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [116]:
ts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,omim_genes
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc...","GJA5,GJA8"
62,chr15,22832519,28379874,ISCA-37404,"15q11q13 recurrent (PWS/AS) region (BP1-BP3, C...","NIPA1,MKRN3,MAGEL2,NDN,SNRPN,UBE3A,GABRB3,GABR..."
23,chr15,23747996,28379874,ISCA-37478,"15q11q13 recurrent (PWS/AS) region (BP2-BP3, C...","MKRN3,MAGEL2,NDN,SNRPN,UBE3A,GABRB3,GABRA5,OCA2"
64,chr16,29649997,30199852,ISCA-37400,"16p11.2 recurrent region (proximal, BP4-BP5) (...","KIF22,PRRT2,ALDOA,TBX6"
47,chr17,1247833,2588909,ISCA-37430,17p13.3 (Miller-Dieker syndrome) region (inclu...,"INPP5K,PRPF8,WDR81,SERPINF2,SERPINF1,DPH1,PAFA..."


In [117]:
ts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/ts-region-20200109.sorted.bed', sep='\t', index=False)

In [118]:
!bgzip -cf data/ts-region-20200109.sorted.bed > ../data/ts-region-20200109.sorted.bed.gz

In [119]:
!tabix -fp bed ../data/ts-region-20200109.sorted.bed.gz

In [120]:
uts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [121]:
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region)
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region)
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere
33,chr15,22832519,23090897,ISCA-37448,15q11.2 recurrent region (BP1-BP2) (includes N...
5,chr15,32019621,32445405,ISCA-46295,15q13.3 recurrent region (D-CHRNA7 to BP5) (in...


In [122]:
uts_region['genes'] = uts_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom']) & (gene['txStart'] >= row['start']) & (gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [123]:
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region),"FAM72C,PPIAL4C,FCGR1A"
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region),"OR4E2,OR4E1"
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere,
33,chr15,22832519,23090897,ISCA-37448,15q11.2 recurrent region (BP1-BP2) (includes N...,"TUBGCP5,CYFIP1,NIPA2,NIPA1"
5,chr15,32019621,32445405,ISCA-46295,15q13.3 recurrent region (D-CHRNA7 to BP5) (in...,


In [124]:
uts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/uts-region-20200109.sorted.bed', sep='\t', index=False)

In [125]:
!bgzip -cf data/uts-region-20200109.sorted.bed > ../data/uts-region-20200109.sorted.bed.gz

In [126]:
!tabix -fp bed ../data/uts-region-20200109.sorted.bed.gz

In [127]:
dgv = pd.read_csv(
    'data/DGV.GS.March2016.50percent.GainLossSep.Final.hg19.gff3', sep='\t', names=['chrom', 'info'], usecols=[0, 8]
).drop_duplicates('info')
info = dgv['info'].str.extract(
    r'ID=(?P<id>[^;]+).*variant_sub_type=(?P<type>[^;]+).*outer_start=(?P<start>[^;]+).*outer_end=(?P<end>[^;]+).*num_samples=(?P<sample>[^;]+).*Frequency=(?P<freq>\S+?)%;'
).astype({'start': int, 'end': int, 'sample': int, 'freq': float})
dgv = dgv.merge(info, left_index=True, right_index=True)
dgv = dgv[(dgv['freq'] >= 1) & (dgv['sample'] >= 10)].sort_values(['chrom', 'start', 'end'])

In [128]:
dgv.head()

,chrom,info,id,type,start,end,sample,freq
33,chr1,ID=gssvG19;Name=gssvG19;variant_type=CNV;varia...,gssvG19,Gain,10001,82189,13,19.40
24,chr1,ID=gssvG13;Name=gssvG13;variant_type=CNV;varia...,gssvG13,Gain,10001,127330,30,44.78
12,chr1,ID=gssvG9;Name=gssvG9;variant_type=CNV;variant...,gssvG9,Gain,10499,235938,13,1.13
93,chr1,ID=gssvG53;Name=gssvG53;variant_type=CNV;varia...,gssvG53,Gain,21064,35983,12,30.00
90,chr1,ID=gssvG50;Name=gssvG50;variant_type=CNV;varia...,gssvG50,Gain,70007,88084,11,27.50


In [129]:
def fetch_gene(gene, chrom, start, end):
    return ','.join(gene.loc[
        (gene['chrom'] == chrom) & (gene['txStart'] >= start) & (gene['txEnd'] <= end), 'name2'
    ])

In [130]:
with Pool(processes=7) as pool:
    dgv['genes'] = pool.starmap(fetch_gene, (
        (gene, row['chrom'], row['start'], row['end']) for _, row in dgv.iterrows()
    ), chunksize=70)

In [131]:
dgv.head()

,chrom,info,id,type,start,end,sample,freq,genes
33,chr1,ID=gssvG19;Name=gssvG19;variant_type=CNV;varia...,gssvG19,Gain,10001,82189,13,19.40,OR4F5
24,chr1,ID=gssvG13;Name=gssvG13;variant_type=CNV;varia...,gssvG13,Gain,10001,127330,30,44.78,OR4F5
12,chr1,ID=gssvG9;Name=gssvG9;variant_type=CNV;variant...,gssvG9,Gain,10499,235938,13,1.13,OR4F5
93,chr1,ID=gssvG53;Name=gssvG53;variant_type=CNV;varia...,gssvG53,Gain,21064,35983,12,30.00,
90,chr1,ID=gssvG50;Name=gssvG50;variant_type=CNV;varia...,gssvG50,Gain,70007,88084,11,27.50,


In [132]:
dgv.loc[dgv['type'] == 'Gain', ['chrom', 'start', 'end', 'id', 'genes']
].rename(columns={'chrom': '#chrom'}).to_csv('data/dgv-gain.sorted.bed', sep='\t', index=False)

In [133]:
!bgzip -cf data/dgv-gain.sorted.bed > ../data/dgv-gain.sorted.bed.gz

In [134]:
!tabix -fp bed ../data/dgv-gain.sorted.bed.gz

In [135]:
dgv.loc[dgv['type'] == 'Loss', ['chrom', 'start', 'end', 'id', 'genes']
].rename(columns={'chrom': '#chrom'}).to_csv('data/dgv-loss.sorted.bed', sep='\t', index=False)

In [136]:
!bgzip -cf data/dgv-loss.sorted.bed > ../data/dgv-loss.sorted.bed.gz

In [137]:
!tabix -fp bed ../data/dgv-loss.sorted.bed.gz

In [138]:
gnomad = pd.read_csv(
    'data/gnomad_v2.1_sv.controls_only.sites.bed.gz', sep='\t', usecols=[0, 1, 2, 3, 4, 241], dtype=str
)
gnomad = gnomad[(gnomad['FILTER'] == 'PASS') & gnomad['svtype'].isin(['DEL', 'DUP'])]
gnomad['#chrom'] = 'chr' + gnomad['#chrom']
gnomad['start'] = gnomad['start'].astype(int)
gnomad['end'] = gnomad['end'].astype(int)

In [139]:
gnomad.head()

,#chrom,start,end,name,svtype,FILTER
6,chr1,62398,62489,gnomAD-SV_v2.1_DEL_1_3,DEL,PASS
7,chr1,66349,66427,gnomAD-SV_v2.1_DEL_1_4,DEL,PASS
15,chr1,136999,155500,gnomAD-SV_v2.1_DUP_1_3,DUP,PASS
16,chr1,141474,155000,gnomAD-SV_v2.1_DUP_1_4,DUP,PASS
24,chr1,319999,327650,gnomAD-SV_v2.1_DEL_1_9,DEL,PASS


In [140]:
with Pool(processes=7) as pool:
    gnomad['genes'] = pool.starmap(fetch_gene, (
        (gene, row['#chrom'], row['start'], row['end']) for _, row in gnomad.iterrows()
    ), chunksize=70)

In [141]:
gnomad.head()

,#chrom,start,end,name,svtype,FILTER,genes
6,chr1,62398,62489,gnomAD-SV_v2.1_DEL_1_3,DEL,PASS,
7,chr1,66349,66427,gnomAD-SV_v2.1_DEL_1_4,DEL,PASS,
15,chr1,136999,155500,gnomAD-SV_v2.1_DUP_1_3,DUP,PASS,
16,chr1,141474,155000,gnomAD-SV_v2.1_DUP_1_4,DUP,PASS,
24,chr1,319999,327650,gnomAD-SV_v2.1_DEL_1_9,DEL,PASS,


In [142]:
gnomad.loc[gnomad['svtype'] == 'DEL', ['#chrom', 'start', 'end', 'name', 'genes']
].to_csv('data/gnomad-del.sorted.bed', sep='\t', index=False)

In [143]:
!bgzip -cf data/gnomad-del.sorted.bed > ../data/gnomad-del.sorted.bed.gz

In [144]:
!tabix -fp bed ../data/gnomad-del.sorted.bed.gz

In [145]:
gnomad.loc[gnomad['svtype'] == 'DUP', ['#chrom', 'start', 'end', 'name', 'genes']
].to_csv('data/gnomad-dup.sorted.bed', sep='\t', index=False)

In [146]:
!bgzip -cf data/gnomad-dup.sorted.bed > ../data/gnomad-dup.sorted.bed.gz

In [147]:
!tabix -fp bed ../data/gnomad-dup.sorted.bed.gz